In [26]:
import pandas as pd
from datasets import Dataset
df1 = pd.read_parquet("testdata_nondepressed.parquet")
df2 = pd.read_parquet("testdata_depression.parquet")

# Append df2 to df1
appended_df = pd.concat([df1,df2])

dataset = Dataset.from_pandas(appended_df)

In [27]:
appended_df.head(2)

,label,value
0,0,I just recently lost my 91 year old grandmothe...
1,0,I don't want to be alive but i'm never gonna k...


In [28]:
from datasets import load_dataset

#dataset = load_dataset("yelp_review_full")
#dataset["train"][100]

In [29]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("mental/mental-bert-base-uncased")


def tokenize_function(examples):
    return tokenizer(examples["value"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/3412 [00:00<?, ? examples/s]

In [30]:
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# Training dataset
train_data = appended_df.sample(frac=0.8, random_state=42)

# Testing dataset
test_data = appended_df.drop(train_data.index)
small_train_dataset = Dataset.from_pandas(train_data)
small_eval_dataset = Dataset.from_pandas(test_data)

In [31]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("mental/mental-bert-base-uncased", num_labels=2)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at mental/mental-bert-base-uncased and are newly initialized: ['bert.pooler.dense.weight', 'classifier.weight', 'bert.pooler.dense.bias', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [33]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [34]:
from transformers import TrainingArguments, Trainer

#training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

# Set up training arguments
training_args = TrainingArguments(
    output_dir="./classification_transfer_learning_transformer/",          
    logging_dir='./classification_transfer_learning_transformer/logs',            
    logging_strategy='epoch',
    logging_steps=100,    
    num_train_epochs=2,              
    #per_device_train_batch_size=32,  
    #per_device_eval_batch_size=32,  
    learning_rate=5e-6,
    seed=42,
    save_strategy='epoch',
    save_steps=100,
    evaluation_strategy='epoch',
    eval_steps=100,
    load_best_model_at_end=True
)

In [35]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

In [36]:
trainer.train()

c:\Users\Mirel Agy\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/684 [00:00<?, ?it/s]

ValueError: You have to specify either input_ids or inputs_embeds